In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import TextVectorization
import numpy as np
import os
import re
import string
import random
import pandas as pd


2024-05-03 15:31:56.440216: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 15:31:57.697996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.list_physical_devices('GPU')

2024-05-03 15:31:58.836535: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 15:31:58.837122: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

<h1>Data Preperation</h1> 	

In [3]:
#Import data
# data = pd.read_csv("./Trump-Tweets/Donald-Tweets!.csv")
# data
df1 = pd.read_csv("./Trump-Tweets/realdonaldtrump.csv")
df2 = pd.read_csv("./Trump-Tweets/trumptweets.csv")
data = pd.concat([df1,df2])

In [4]:
data

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
41117,1218962544372670467,https://twitter.com/realDonaldTrump/status/121...,I have never seen the Republican Party as Stro...,2020-01-19 19:24:52,32620,213817,NaN,NaN,NaN
41118,1219004689716412416,https://twitter.com/realDonaldTrump/status/121...,Now Mini Mike Bloomberg is critical of Jack Wi...,2020-01-19 22:12:20,36239,149571,NaN,NaN,NaN
41119,1219053709428248576,https://twitter.com/realDonaldTrump/status/121...,I was thrilled to be back in the Great State o...,2020-01-20 01:27:07,16588,66944,NaN,#,NaN
41120,1219066007731310593,https://twitter.com/realDonaldTrump/status/121...,"“In the House, the President got less due proc...",2020-01-20 02:16:00,20599,81921,@ @ @,NaN,NaN


In [5]:
#Get Tweets
tweet_text_raw = data['content']
# tweet_text_raw = data['Tweet_Text']
print(tweet_text_raw)

0        Be sure to tune in and watch Donald Trump on L...
1        Donald Trump will be appearing on The View tom...
2        Donald Trump reads Top Ten Financial Tips on L...
3        New Blog Post: Celebrity Apprentice Finale and...
4        "My persona will never be that of a wallflower...
                               ...                        
41117    I have never seen the Republican Party as Stro...
41118    Now Mini Mike Bloomberg is critical of Jack Wi...
41119    I was thrilled to be back in the Great State o...
41120    “In the House, the President got less due proc...
41121    A great show! Check it out tonight at 9pm. @ F...
Name: content, Length: 84474, dtype: object


In [6]:
#Remove http links
tweet_text_no_https = tweet_text_raw.str.replace(r'http\S+', '')
#Remove links without http
tweet_text_no_https = tweet_text_no_https.str.replace(r'\s*[^ /]+/[^ /]+', '')
#Set all empty strings to nan
tweet_text_no_https.replace('', np.nan, inplace=True)
#drop nan rows
tweet_text_no_links = tweet_text_no_https.dropna()
#Remove space after @
tweet_text_better_mentions = tweet_text_no_links.str.replace(r'@\s+', '@')
#tweet_text_no_mentions = tweet_text_no_links.str.replace(r'@\w+', '')
#Remove space after #
tweet_text_better_hashtags = tweet_text_better_mentions.str.replace(r'#\s', '#')
#Set short tweets to NaN and drop them
tweets_long = tweet_text_better_hashtags.apply(lambda x: x if len(x) > 40 else np.NaN)
tweets_long = tweets_long.dropna()
#remove emojis
tweets_no_emojis = tweets_long.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
tweets_lower_case = tweets_no_emojis.str.lower()
#remove unwanted punctuation
tweet_text_no_unwanted_punctuation = tweets_lower_case.str.replace(r"[\"$%()*+,-/:;<>=\[\]\\_`{}\|~.]", '')

tweet_text_clean = tweet_text_no_unwanted_punctuation
#tweet_text_clean.to_csv('./Trump-Tweets/clean_data.csv')
tweet_text_clean[10]

/tmp/ipykernel_315493/3662983176.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_text_no_https = tweet_text_raw.str.replace(r'http\S+', '')
/tmp/ipykernel_315493/3662983176.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_text_no_https = tweet_text_no_https.str.replace(r'\s*[^ /]+/[^ /]+', '')
/tmp/ipykernel_315493/3662983176.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_text_better_mentions = tweet_text_no_links.str.replace(r'@\s+', '@')
/tmp/ipykernel_315493/3662983176.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_text_better_hashtags = tweet_text_better_mentions.str.replace(r'#\s', '#')
/tmp/ipykernel_315493/3662983176.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_text_no_unwanted

10    dont be afraid of being unique  it's like bein...
10    dont be afraid of being unique  it's like bein...
Name: content, dtype: object

In [7]:
#https://keras.io/examples/generative/text_generation_with_miniature_gpt/

In [8]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)


In [9]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [10]:
#Average length of a tweet
max_length = max([len(tweet.split(' ')) for tweet in tweet_text_clean])
max_length

59

In [11]:
tweets = []
for tweet in tweet_text_clean:
    sequence = []
    for word in tweet.split(' '):
        tweets.append(word)
len(tweets)

1617697

In [12]:
vocab_size = 20000
maxlen = max_length  # Max sequence size
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model

In [13]:
batch_size = 128

text_ds = tf.data.Dataset.from_tensor_slices(tweet_text_clean)
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)


In [14]:

# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen + 1,
    standardize= None,#'lower_and_strip_punctuation',
    split='whitespace',
)

In [15]:
vectorize_layer.adapt(tweet_text_clean)
vocab = vectorize_layer.get_vocabulary()  # To get words back from token indices
len(vocab)

19999

In [16]:
def prepare_lm_inputs_labels(text):
    """
    Shift word sequences by 1 position so that the target for position (i) is
    word at position (i+1). The model will use all words up till position (i)
    to predict the next word.
    """
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

In [17]:
text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

In [18]:
class TextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
       
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x,verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")
        
    def generate_text(self, start_string_tokens) -> str:
        start_tokens = [_ for _ in start_string_tokens]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x,verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            #get [UNK] -> vocab[1]
            if sample_token == 1:
                continue
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join([self.detokenize(_) for _ in start_tokens + tokens_generated])
        return txt

In [19]:
# Tokenize starting prompt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

In [20]:
start_prompt = "fake news"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40
text_gen = TextGenerator(num_tokens_generated, start_tokens, vocab)

In [21]:
model = create_model()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 59)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 59, 256)        │     5,135,104 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 59, 256)        │       658,688 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 59, 20000)      │     5,140,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,933,792 (41.71 MB)

 Trainable params: 10,933,792 (41.71 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
tf.config.list_physical_devices('GPU')

[]

In [68]:
NUM_EPOCHS = 25
model.fit(text_ds, verbose=1, epochs=NUM_EPOCHS, callbacks=[text_gen])

Epoch 1/25


2024-05-03 15:28:21.796290: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 604160000 exceeds 10% of free system memory.


In [ ]:
start_prompt = "America is the"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
text_gen.generate_text(start_tokens)

'obamacare does not work 21 tax cut taxes each it is a made is taking in a tremendous cont                           21 tax cut taxes each it is a made is taking in a tremendous cont                          '

In [ ]:
# Save the entire model as a SavedModel.
# model.save('./saved_model/miniature_gpt_model_200_epochs_no_dropout')

INFO:tensorflow:Assets written to: ./saved_model/miniature_gpt_model_200_epochs_no_dropout/assets


INFO:tensorflow:Assets written to: ./saved_model/miniature_gpt_model_200_epochs_no_dropout/assets


In [ ]:
# loading the saved model
# loaded_model = tf.keras.models.load_model('./MyModel_tf')